In [1]:
import os

In [2]:
!pwd

/official/official_docs/PythonLearning/DeepLearning/nlp/Projects/TextSummerizerEndToEnd/research


In [3]:
os.chdir("/official/official_docs/PythonLearning/DeepLearning/nlp/Projects/TextSummerizerEndToEnd")
!pwd

/official/official_docs/PythonLearning/DeepLearning/nlp/Projects/TextSummerizerEndToEnd


In [4]:
# entity
from dataclasses import dataclass
from pathlib import Path

In [10]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    DataIngestion is a dataclass that holds the information needed to ingest data into the database.
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
# configure
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml,create_directories

In [11]:
class ConfigManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [14]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size


In [18]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info:\n{headers})")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}, skipping download")

    def extract_zip_file(self):
        """ 
        Extracts the zip file to the local directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")

In [19]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-10-12 12:21:39,779: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-12 12:21:39,781: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-12 12:21:39,783: INFO: common: created directory at: artifacts]
[2023-10-12 12:21:39,784: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-12 12:21:44,165: INFO: 3069630809: artifacts/data_ingestion/data.zip downloaded with following info:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F81C:7FE5:37B64A:443C21:652796A6
Accept-Ranges: bytes
Date: Thu, 12 Oct 2023 06:51:40 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10234-MAA
X-Cache: HIT